## Collecting Data using APIs

As a first step, we register a free account on the https://data.nasdaq.com/ website.

After we register, we will be provided with a unique API key, that we should store.

Note: Use a .env file and put our key in there and python-dotenv to access it in this notebook.

The key is stored in a .env file, which is added to gitignore.


In [1]:
# get api key from the .env file
import os
from dotenv import load_dotenv  

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')


In [3]:
# First, import the relevant modules
import json
import requests


### Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017.

In [6]:
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json'
params = {'api_key' : API_KEY, 'start_date' : '2017-01-01', 'end_date' : '2017-12-31' }

r = requests.get(url, params=params, stream=True)

#r.url
#r.status_code
#r.headers['content-type']
#r.json()

#r.text
#print(r.content)

#r.raw



In [ ]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

r.json()

In [98]:

print(type(r.json()))

<class 'dict'>


In [199]:
# Convert the returned JSON object into a Python dictionary.
fse_data = r.json()
#print(fse_data['dataset_data'])
#print(type(fse_data['dataset_data']['column_names']))
print(fse_data['dataset_data']['column_names'])
#print(type(fse_data['dataset_data']['data']))
#print(fse_data['dataset_data']['data'][0])




['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']


### Calculate what the highest and lowest opening prices were for the stock in this period.

In [152]:
# Make a list of opening prices
m = len(fse_data['dataset_data']['data'])
opening = []
for i in range(m):
    opening.append(fse_data['dataset_data']['data'][i][1])    

In [158]:
# Removing None values in opening price list
opening = list(filter(lambda item: item is not None, opening))

In [180]:
# Find the highest and lowest opening prices
highest_open = max(opening)
lowest_open = min(opening)


### Calculating the largest change in any one day (based on High and Low price)

In [159]:
# Make a list of daily high prices

m = len(fse_data['dataset_data']['data'])
high = []
for i in range(m):
    high.append(fse_data['dataset_data']['data'][i][2])    

In [181]:
# Make a list of daily low prices

m = len(fse_data['dataset_data']['data'])
low = []
for i in range(m):
    low.append(fse_data['dataset_data']['data'][i][3])  

In [198]:
# Make a list of daily changes according to the daily high and low prices
change_high_low = []
for item1, item2 in zip(high, low):
    item = item1 - item2
    change_high_low.append(item)

# Finding the highest change   
max_change_high_low = max(change_high_low)
print("The highest daily change is:", max_change_high_low) 

The highest daily change is: 2.8100000000000023


### Calculate the largest change between any two days (based on Closing Price)

In [170]:
# Make a list of daily closing prices

m = len(fse_data['dataset_data']['data'])
close = []
for i in range(m):
    close.append(fse_data['dataset_data']['data'][i][4])  

In [176]:
# Make a list of 2 day changes according to daily closing prices
change_close = []
for i in range(m-1):
    item = abs(close[i+1] - close[i])
    change_close.append(item)

In [197]:
# Finding the highest change 
max_change_close = max(change_close)
print("The highest 2-day change is:", max_change_close)

The highest 2-day change is: 2.559999999999995


### Calculate the average daily trading volume during that year

In [183]:
# Make a list of daily trading volumes

m = len(fse_data['dataset_data']['data'])
trade_volume = []
for i in range(m):
    trade_volume.append(fse_data['dataset_data']['data'][i][6])  

In [195]:
# Define a function to compute the average of a list

def avg(lst):
    return sum(lst) / len(lst)

# Compute the average daily traing volume

avg_trade_vol = avg(trade_volume)
print("The average daily trade volume is:" , avg_trade_vol)

The average daily trade volume is: 89124.33725490196


### Calculate the median trading volume during that year. 

In [196]:
trade_volume.sort()
mid = len(trade_volume) // 2
med_trade_vol = (trade_volume[mid] + trade_volume[~mid]) / 2

print("The median daily trade volume is:" , med_trade_vol)

The median daily trade volume is: 76286.0
